### <b> Task-Adaptive Meta Learning algorithm (TAML) </b>

In [44]:
"""setup information"""
import torch
import matplotlib.pyplot as plt
import TAML_code
import Data
import importlib as imp
import numpy as np
import encoder
"""dataset split as specified by the authors"""
train_class_names = [
    'train', 'skyscraper', 'turtle', 'raccoon', 'spider', 'orange', 'castle', 'keyboard',
    'clock', 'pear', 'girl', 'seal', 'elephant', 'apple', 'aquarium_fish', 'bus',
    'mushroom', 'possum', 'squirrel', 'chair', 'tank', 'plate', 'wolf', 'road', 'mouse',
    'boy', 'shrew', 'couch', 'sunflower', 'tiger', 'caterpillar', 'lion', 'streetcar',
    'lawn_mower', 'tulip', 'forest', 'dolphin', 'cockroach', 'bear', 'porcupine', 'bee',
    'hamster', 'lobster', 'bowl', 'can', 'bottle', 'trout', 'snake', 'bridge',
    'pine_tree', 'skunk', 'lizard', 'cup', 'kangaroo', 'oak_tree', 'dinosaur', 'rabbit',
    'orchid', 'willow_tree', 'ray', 'palm_tree', 'mountain', 'house', 'cloud'
    ]
valid_class_names = [
    'otter', 'motorcycle', 'television', 'lamp', 'crocodile', 'shark', 'butterfly', 'sea',
    'beaver', 'beetle', 'tractor', 'flatfish', 'maple_tree', 'camel', 'crab', 'cattle'
    ]
test_class_names = [
    'baby', 'bed', 'bicycle', 'chimpanzee', 'fox', 'leopard', 'man', 'pickup_truck',
    'plain', 'poppy', 'rocket', 'rose', 'snail', 'sweet_pepper', 'table', 'telephone',
    'wardrobe', 'whale', 'woman', 'worm'
    ]

In [ ]:
def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
force_cudnn_initialization()
imp.reload(TAML_code)
imp.reload(encoder)
number_of_inner_gradient_steps: int = 5
ways: int = 5
max_shots: int = 50
query: int = 15
iterations = 1000 # epochs
inner_learning_rate: float = 0.5
outer_learning_rate: float = 1e-2
batch_size = 4 # how many tasks in this batch
total_train_task = batch_size * iterations
total_valid_task = batch_size * int(iterations * 0.16)
total_test_task = 600
m = TAML_code.TAML(output_dimension=ways, max_shots=max_shots,
    inner_loop_step_ct=number_of_inner_gradient_steps,
    inner_learning_rate=inner_learning_rate, learn_inner_learning_rate=True, 
    outer_learning_rate=outer_learning_rate, batch_size=batch_size,
    encoder=encoder.InferenceNetwork(ways, max_shots, 'cifar', True, True, True))
# m._view_model(m._meta_parameters)
train_loss, train_accuracy, valid_loss, valid_accuracy = m.train(
    dataloader_train=Data.get_dataloader(train_class_names, max_shots, query, ways, total_train_task, batch_size),
    dataloader_valid=Data.get_dataloader(valid_class_names, max_shots, query, ways, total_valid_task, batch_size))
test_loss, test_accuracy = m.test( 
    dataloader_test =Data.get_dataloader(test_class_names,  max_shots, query, ways, total_valid_task, batch_size))
# plot results:
f, ax = plt.subplots(figsize=(8, 8), nrows=2, ncols=2)
# plot train
ax[0, 0].plot(train_loss)
ax[0, 0].set_title("train loss")
ax[1, 0].plot(train_accuracy)
ax[1, 0].set_title("train accuracy")
# plot valid
ax[0, 1].plot(valid_loss)
ax[0, 1].set_title("valid loss")
ax[1, 1].plot(valid_accuracy)
ax[1, 1].set_title("valid accuracy")